Here is a simple example of how the package can be used and what requirements must be met in order to be able to use the full range of functions.
A basic requirement is the availability of downloaded satellite image data (so far only Sentinel-2 is supported). The data structure should look similar to this: 

![Example data structure](./img/example_data_structure.png)

The most important thing is that the bands in the folder are marked with a "B" and the number and stored as GeoTIFF, e.g. ..._B02.tif, ..._B12.tif. If the data is to be pre-processed for a deep learning model, then a file with the corresponding labels must also be provided for each image and contain the name annotation (see example: annotation.tif).

In [1]:
# Make sure the package is installed
# pip install satellite_datacube

In [1]:
from satellite_datacube.datacube import SatelliteDataCube 

# Define folder of satellite data and specify some parameters (read more in the doc-string)
satellite = "S2" # or S1 or S12
base_folder = r"D:\SatelliteDataCube\Chimanimani" # path_of_SITS = ".../Sentinel2"
S2_Chimanimani = SatelliteDataCube(satellite=satellite, base_folder=base_folder)


----------------------------------------
Initialized data-cube with following parameter:
- Images from satellite mission: S2
- Base folder: D:\SatelliteDataCube\Chimanimani
- Start-End: 2018-09-06 00:00:00 -> 2019-09-16 00:00:00
- Length of data-cube: 76
----------------------------------------


In [ ]:
# Make a selection of images and store them as timeseries 
ts10 = S2_Chimanimani.create_timeseries(timeseries_length=10)
print(ts10)

Selecting timeseries of length 6 with bad pixel limit of 15 % for each satellite image
None 15


TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [ ]:
# Show global data
S2_datacube.global_data

{'global_mask': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]]),
 'ts-6': array([[       0,       15,       28,       47,       60,       74],
        [20180906, 20181120, 20190124, 20190429, 20190703, 20190911]])}

In [ ]:
# Add timeseries to global data file 
S2_datacube.create_global_data(timeseries_length=8, bad_pixel_limit=S2_datacube.parameters["badPixelLimit"])
S2_datacube.global_data

Building global mask of datacube
Selecting timeseries of length 8 with bad pixel limit of 15 % for each satellite image
Saving global data in: D:\SatelliteDataCube\Chimanimani\global_data.hdf5


{'global_mask': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]]),
 'ts-6': array([[       0,       15,       28,       47,       60,       74],
        [20180906, 20181120, 20190124, 20190429, 20190703, 20190911]]),
 'ts-8': array([[       0,       10,       21,       34,       42,       55,
               64,       74],
        [20180906, 20181026, 20181220, 20190223, 20190404, 20190608,
         20190723, 20190911]])}

In [ ]:
S2_datacube.preprocess()

Loading global data from: D:\SatelliteDataCube\Chimanimani\global_data.hdf5
Loading patches as dictonary with keys [img, msk, msk_gb] from .npy files
